In [1]:
import math
import struct
from encodings.johab import codec
from http.cookiejar import offset_from_tz_string
from os import name


# First Generation Lunar Use
class SPDU4:
    # Class attributes here: shared constants

    """Link Establishment and Control Directive"""
    LEC_FIELDS = [ # (name, bit_start, bit_length)
        ("directive_name", 0, 3),
        ("link_direction", 3, 1),
        ("demand_query", 4, 1),
        ("query_response", 5, 1),
        ("remote_no_more_data", 6, 1),
        ("token", 7, 1),
        ("duplex_simplex", 8, 3),
        ("frequency", 11, 5),
        ("polarization", 16, 1),
        ("modulation_index", 17, 3),
        ("modulation", 20, 4),
        ("spares", 24, 2),
        ("coding", 26, 6),
        ("link_snr", 32, 8),
        ("symbol_rate", 40, 16),
    ]

    DIRECTIVE_NAMES_MAP = {
        0b000: "Link Establishment & Control",
        0b001: "Report Request",
        0b010: "Set V(R)",
        0b011: "Report Source Spacecraft ID",
        0b100: "PN Ranging",
        # 0b100 to 0b111: Reserved
    }

    LINK_DIRECTION_MAP = {
        0b0: "Return Link",
        0b1: "Forward Link",
    }

    DEMAND_QUERY_MAP = {
        0b0: "ACK",
        0b1: "NACK",
    }

    RNMD_MAP = {
        0b0: "No Change",
        0b1: "Remote Node has No More Data to Send (RNMD)",
    }

    TOKEN_MAP = {
        0b0: "No Change",
        0b1: "Transmit",
    }

    DUPLEX_SIMPLEX_MAP = {
        0b000: "No Change",
        0b001: "Full Duplex",
        0b010: "Half Duplex",
        0b011: "Simplex Transmit",
        0b100: "Simplex Receive",
        # 0b101-0b111: Reserved
    }

    # Forward: 5-bit binary code -> channel (0-31)
    FORWARD_FREQUENCY_MAP = {i: f"Forward Channel {i}" for i in range(32)}
    # Return: 5-bit binary code -> channel (0-31)
    RETURN_FREQUENCY_MAP = {i = f"Return Channel {i}" for i in range(32)}

    POLARIZATION_MAP = {
        0b0: 'Left Hand CP',
        0b1: 'Right Hand CP'
    }

    MODULATION_INDEX_MAP = {
        0b000: 0.0,
        0b001: 0.4,
        0b010: 0.6,
        0b011: 0.8,
        0b100: math.pi/3,
        0b101: 1.15,
        0b110: 1.3,
        0b111: 1.4,
    }

    MODULATION_MAP = {
        0b0000: "PCM/PM/Bi-phase-L (filtered)",
        0b0001: "GMSK",
        # Others reserved
    }

    CODING_MAP = {
        0b000000: "Uncoded",
        0b000001: "LDPC(2048, 1024)",
        0b000101: "LDPC(6144, 4096)",
        0b001010: "LDPC(8160, 7136)",
        # Others reserved
    }

    """ Report Request """
    RR_FIELDS = [ # (name, bit_start, bit_length)
        ("directive_name", 0, 3),
        ("pcid0_plcw_request", 3, 1),
        ("pcid1_plcw_request", 4, 1),
        ("time_tag_sample_request", 5, 6),
        ("status_report_request", 11, 5),
    ]

    """ Set V(R) """
    SET_VR_FIELDS = [
        ("directive_name", 0, 3),
        ("spare", 3, 5),
        ("seq_ctrl_fsn", 8, 8),
    ]

    """ Report Source Spacecraft ID"""
    REPORT_SOURCE_SCID_FIELDS = [
        ("directive_name", 0, 3),
        ("reserved", 3, 13),
        ("source_SCID", 16, 16),
    ]

    """ PN Ranging """
    PN_RANGING_FIELDS = [
        ("directive_name", 0, 3),
        ("mode_type", 3, 2),
        ("ranging_code", 5, 2),
        ("chip_rate_k", 7, 3),
        ("chip_rate_l", 10, 14),
        ("chip_rate_m", 24, 14),
        ("ranging_mod_index", 38, 3),
        ("pn_epoch_time_tag_day", 41, 16),
        ("pn_epoch_time_tag_ms_of_day", 57, 32),
        ("status_report_request", 89, 5),
        ("spares", 94, 2),
    ]

    def __init__(self):
        self.directive_name = None

    def handle_directive(self, directive_code: int, payload: bytes = b''):
        """Dispatch to directive logic"""
        self.directive_name = DIRECTIVE_NAMES_MAP.get(directive_code, "Reserved")
        match directive_code:
            case 0b0000:
                self._link_establishment_control(payload)
            case 0b001:
                self._report_request(payload)
            case 0b010:
                self._set_vr(payload)
            case 0b011:
                self._report_source_scid(payload)
            case 0b100:
                self._pn_ranging(payload)
            case _:
                print(f"Reserved directive: {directive_code:03b}")

    def _link_establishment_control(self, payload: bytes):
        self.link direction = 0
        if
        demand/query
        query response
        rnmd
        token
        duplex/simplex
        frequency
        polarization
        mod index
        modulation
        spares
        coding
        instantaneous link SNR
        symbol rate
    def _report_request(self, payload: bytes):
        directive name
        pcid 0 plcw request
        pcid 1 plcw request
        time tag sample request
        status report request
    def _set_vr(self, payload: bytes):
        """Update V(R) from bits."""
        directive name
        receiver frame sequence number seq ctrl fsn
    def _report_scid(self, payload: bytes):
        """Extract SCID from bits"""
        directive name
        reserved
        source spacecraft ID
    def _pn_ranging(self, payload: bytes):
        directive name
        mode type
            ranging off
            one-way ranging
            two-way non-regenerative ranging
            two-way regenerative ranging
        ranging code

        chip rate (k,l,m)
        ranging mod index
        pn epoch time tag
        status report request
        spares


    LEC directive
        directive name
        [000] for LEC
        link direction
        [0] for return
        [1] for forward
        demand/query
        [0] for demand (command)
        [1] for query (link negotiation)
        query response
        [0] for ack
        [1] for nack
        remote no more data
        [0] for no change
        [1] for remote node has no more data to send
        token
        [0] for no change
        [1] for transmit
        duplex/simplex
        [bbb] DUPLEX_SIMPLEX_MAP
        frequency
        - forward link
        - return link
        polarization
        [0] for LHCP
        [1] for RHCP
        modulation index
        [bbb] for MODULATION_INDEX
        modulation
        [bbbb] for MODULATION_MAP
        spares
        coding
        instantaneous link snr
        symbol rate
    Report request
    Set vr
    report source id
    pn ranging
    reserved

SyntaxError: unmatched '}' (2870039683.py, line 41)